In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import time

# Plan de vol

Certains vols peuvent engendrer des conflits potentiels : c’est par exemple le cas lorsque deux avions
suivent un même trajet, en sens inverse, le même jour et à un même niveau. Écrire une requête SQL qui fournit
la liste des couples (Id 1 , Id 2 ) des identifiants des vols dans cette situation.

La requete suivante fonctionne mais retourne les couples (Id1, Id2) et (Id2, Id1)

Il suffit d'ordonner le couple (Id1, Id2) pour dédoublonner

# II Allocation des niveaux de vol

In [2]:
#conflit est une variable globale

n = 3
conflit = np.random.randint(0, 501, size = (3*n, 3*n) )
conflit = np.array([[i > j and conflit[j][i] or i < j and conflit[i][j] for j in range(3*n) ] for i in range(3*n) ])
for i in range(0, len(conflit), 3):
    conflit[i:i+3,i:i+3] = np.array([[0]*3 for _ in range(3)])
conflit

array([[  0,   0,   0,  41, 399, 281, 357, 186, 229],
       [  0,   0,   0, 497, 468, 174, 442, 249, 245],
       [  0,   0,   0, 336, 215, 491, 243, 246,  30],
       [ 41, 497, 336,   0,   0,   0, 340, 344,  20],
       [399, 468, 215,   0,   0,   0, 371, 410, 124],
       [281, 174, 491,   0,   0,   0, 367,  13, 352],
       [357, 442, 243, 340, 371, 367,   0,   0,   0],
       [186, 249, 246, 344, 410,  13,   0,   0,   0],
       [229, 245,  30,  20, 124, 352,   0,   0,   0]])

In [51]:
conflit = [ [  0,  0,  0,100,100,  0,  0,150,  0],
            [  0,  0,  0,  0,  0, 50,  0,  0,  0],
            [  0,  0,  0,  0,200,  0,  0,300, 50],
            [100,  0,  0,  0,  0,  0,400,  0,  0],
            [100,  0,200,  0,  0,  0,200,  0,100],
            [  0, 50,  0,  0,  0,  0,  0,  0,  0],            
            [  0,  0,  0,400,200,  0,  0,  0,  0],            
            [150,  0,300,  0,  0,  0,  0,  0,  0],            
            [  0,  0, 50,  0,100,  0,  0,  0,  0] ]

In [4]:
def nb_conflits():
    n = len(conflit)
    nb = 0
    for i in range(n - 1):
        for j in range(i + 1, n):
            if conflit[i][j] != 0:
                nb += 1
    return nb

In [5]:
nb_conflits()

8

Cette fonction est de complexité temporelle :
\begin{equation*}
3n -1 + 3n- 2 + \ldots + 1 = \frac{(3n-1)3n}{2}=O(n^2)
\end{equation*}
Il s'agit d'une complexité quadratique

In [52]:
def cout_regulation(regulation):
    cout = 0
    n = len(regulation)
    sommet = [3*k + regulation[k] for k in range(n)]
    for k in range(n - 1):
        s = sommet[k]
        for j in range(k + 1, n):            
            cout += conflit[s][sommet[j]]
    return cout   

In [7]:
cout_regulation([0, 1, 1])

250

In [8]:
def nb_vol_par_niveau_relatif(regulation):
    cout = [0]*3
    for r in regulation:
        cout[r] += 1
    return cout

In [9]:
nb_vol_par_niveau_relatif([0, 1, 1])

[1, 2, 0]

La fonction `cout_regulation` effectue $n - 1$ tours de boucles externes.
Chaque tour de boucle externe comprend $n - k - 1$ tours de boucles internes.
Chaque tour de boucle interne comprend une affectation.
La complexité de `cout_regulation` est donc en $n - 1 + n - 2 + \ldots + 1 = \frac{n(n-1)}{2}=O(n^2)$, elle est __quadratique__.

In [10]:
def cout_RFL():
    return cout_regulation([0]*(len(conflit)//3))

In [11]:
cout_RFL()

500

Pour $n$ vols, il existe $3^n$ régulations possibles qui sont les $n-listes$ d'élements pris dans l'ensemble `{0,1,2}`

Il n'est pas envisageable de calculer les couts de toutes les régulations possibles pour trouver celle de cout minimal, car cet algorithme aurait une complexité en $O(n^2 3^n)$.

## Algorithme minimal

In [12]:
def cout_du_sommet(s, etat_sommet):
    cout = 0
    adjacents = conflit[s]
    for k in range(len(etat_sommet)):
        if etat_sommet[k] != 0:
            cout += adjacents[k]
    return cout            

La complexité de la fonction `cout_du_sommet` est __linéaire__, en $3n= O(n)$.

In [13]:
cout_du_sommet(8, [1, 0, 0, 0, 1, 0, 2, 2, 2])

100

In [14]:
def sommet_de_cout_min(etat_sommet):
    """Retourne l'index du sommet de cout minimal parmi les sommets qui 
    n'ont pas encore été choisis ou supprimés"""
    cout_min = None
    index_min = None
    for k in range(len(etat_sommet)):
        s = etat_sommet[k]
        if s == 2:
            c = cout_du_sommet(k, etat_sommet)
            if cout_min == None or c < cout_min:
                cout_min = c
                index_min = k
    return index_min

In [15]:
sommet_de_cout_min([1, 0, 0, 0, 1, 0, 2, 2, 2])

8

La complexité de la fonction `sommet_de_cout_min` est __quadratique__ en $3n \times O(n) = O(n^{2})$.

In [16]:
def minimal():
    m = len(conflit)
    n = m//3
    etat_sommet = [2]*m
    regulation = [0]*n
    for k in range(n):
        indexmin = sommet_de_cout_min(etat_sommet)        
        q = indexmin // 3
        regulation[q] = indexmin%3
        for j in range(3):
            index = 3*q + j
            if index != indexmin:
                etat_sommet[index] = 0
            else:
                etat_sommet[index] = 1
    return regulation     

La complexité de la fonction `minimal` est de $n \times O(n^{2})=O(n^{3})$.

In [17]:
minimal()

[1, 0, 1]

In [18]:
cout_regulation([1, 0, 1])

0

## Recuit simulé

L’algorithme de recuit simulé part d’une régulation initiale quelconque (par exemple la régulation pour laquelle
chacun des avions vole à son RFL) et d’une valeur positive T choisie empiriquement. Il réalise un nombre fini
d’étapes se déroulant ainsi :

* un vol v(k) est tiré au hasard ;
*  on modifie  u(k) en tirant au hasard parmi les deux autres valeurs possibles ;

      • si cette modification diminue le cout de la régulation, cette modification est conservée ;
    
      • sinon, cette modification n’est conservée qu’avec une probabilité p = exp(−Δc/T ), où Δc est l’augmentation de cout liée à la modification de la régulation ;
    
* le paramètre T est diminué d’une certaine quantité.

Écrire en Python une fonction recuit(regulation) qui modifie la liste regulation passée en paramètre en
appliquant l’algorithme du recuit simulé. On fera débuter l’algorithme avec la valeur T = 1000 et à chaque
étape la valeur de T sera diminuée de 1%. L’algorithme se terminera lorsque T < 1.

In [5]:
def recuit(regulation):
    T = 1000
    m = len(conflit)
    n = m//3
    cout = cout_regulation(regulation)
    while T >= 1:        
        k = random.randint(0, n - 1)
        rk = regulation[k]
        mk = [0,1,2]
        del(mk[rk])
        newrk = mk[random.randint(0, 1)]
        regulation[k] = newrk
        newcout = cout_regulation(regulation)
        deltac = newcout - cout
        # mise à jour du cout minimal si le nouveau cout est inférieur
        # sinon on rétablit l'ancienne version 
        if newcout < cout or random.random() < math.exp(-deltac/T):
            cout = newcout   
        else:
            regulation[k] = rk
        T = T * 0.99    #on diminue T de 1 %        
    return regulation

In [57]:
recuit([0,0,0])

[1, 0, 2]

In [59]:
cout_regulation([1, 0, 2])

0

In [4]:
np.log(1/1000) / np.log(0.99)

687.31586483008266

# III Système d’alerte de trafic et d’évitement de collision

In [21]:
class transpondeur():
    debit_binaire = 10**6
    temps_emission = 128*10**(-6)
    bits_debut = 6
    bits_fin = 6
    bits_controle = 4
    bits_emis = debit_binaire*temps_emission
    bits_donnees = bits_emis - bits_debut - bits_fin - bits_controle
    altitude_min = 2000
    altitude_max = 66000
    vitesse_ascensionnelle_min = -5000
    vitesse_ascensionnelle_max = 5000

In [22]:
t = transpondeur()
print(t.bits_donnees)

112.0


Supposons que :
    
* l'altitude est un entier non signé codé sur n bits avec un décalage
de $-2000$, il faut donc $17$ bits pour coder les entiers de $0$ à $66000-2000=64000$

* la vitesse ascensionnelle est un entier signé codé en complément à $2$, il faut $14$ bits pour coder les entiers 
de $-5000$ à $5000$

Il faut donc $17 + 14 = 31$ bits pour coder l'altitude et la la vitesse ascensionnelle. Sachant qu'il faut $24$ bits pour l'identifiant de l'avion, celà nécessite au total $31+24=55$ bits  ce qui est bien inférieur aux $112$ bits disponibles.

Supposons que :
    
* l'altitude et la vitesse ascensionnelle soient des des entiers signés, codés en complément à deux, il faut alors $18$ bits pour coder $66000$ la donnée maximale en valeur absolue qui est de signe positif et on a donc besoin de $2 \times 18 = 36$ bits pour coder la vitesse ascensionnelle et l'altitude 

Sachant qu'il faut $24$ bits pour l'identifiant de l'avion, celà nécessite au total $36+24=60$ bits ce qui est bien inférieur aux $112$ bits disponibles.

In [23]:
def nb_bits_entier(n, signe = True):
    negatif = n < 0
    if negatif:
        n = -n 
    m = 1
    p = 2
    while p <= n:       
        p *= 2
        m += 1        
    if not signe or negatif and p/2 == n:
        return m   
    return m + 1

In [24]:
[nb_bits_entier(k, signe = False) for k in range(9)]

[1, 1, 2, 2, 3, 3, 3, 3, 4]

In [25]:
nb_bits_entier(5000, signe = True)

14

In [26]:
2**12 - 1, 2**13 - 1

(4095, 8191)

In [27]:
nb_bits_entier(66000, signe = True)

18

In [28]:
nb_bits_entier(-128, signe = True)

8

In [29]:
nb_bits_entier(128, signe = True)

9

Après avoir récupéré les informations nécessaires, la fonction acquerir_intrus calcule la position et
la vitesse de l’intrus par rapport à l’avion propre et renvoie une liste de huit nombres :


    [id, x, y, z, vx, vy, vz, t0]
    
où

* id est le numéro d’identification de l’intrus ;
* x, y, z les coordonnées (en mètres) de l’intrus dans un repère orthonormé R 0 lié à l’avion propre ;
* vx, vy, vz la vitesse (en mètres par seconde) de l’intrus dans ce même repère ;
* t0 le moment de la mesure (en secondes depuis un instant de référence).

À des fins d’analyse une fois l’avion revenu au sol, la fonction acquerir_intrus conserve chaque résultat obtenu.
Chaque nombre est stocké sur 4 octets. En supposant que cette fonction est appelé au maximum 100 fois par
seconde, quel est le volume de mémoire nécessaire pour conserver les données de 100 heures de fonctionnement
du TCAS ?
Ce volume de stockage représente-t-il une contrainte technique forte ?

In [30]:
class stockage_tcas():
    vstock_avion = 8*4  #volume de stockage d'un avion en octets
    debit_stock = 100  #debit de stockage en nombre de stockage par seconde
    duree_vol = 100*3600
    vstock_total = duree_vol * debit_stock*vstock_avion

In [31]:
print("{:e}".format(stockage_tcas().vstock_total))

1.152000e+09


Le volume de stockage nécessaire pour $100$ heures de fonctionnement est de $1,15$ To environ ce qui ne constitue pas une contrainte technique forte

## Estimation du CPA

`[id, x, y, z, vx, vy, vz, t0] = acquerir_intrus()`

$\overrightarrow {OG}(t) \left(x + (t - t0) \times vx, y + (t - t0) \times vy, z + (t - t0) \times vz \right)$

On a :
    
\begin{equation*}
{\Vert \overrightarrow {OG}(t)\Vert}^{2}=(x + (t - t0) \times vx)^{2} + (y + (t - t0) \times vy)^{2} + (z + (t - t0) \times vz)^{2} 
\end{equation*}

La fonction polynome du second degré $t \mapsto {\Vert \overrightarrow {OG}(t)\Vert}^{2}$ atteint son minimum en :

\begin{align*}
t_{c}=-\frac{b}{2a} & =-\frac{2(x \times vx + y \times vy + z \times vz)}{2({vx}^2+{vy}^{2}+{vz}^{2})}  \\
t_{c}=-\frac{b}{2a} & =-\frac{x \times vx + y \times vy + z \times vz}{{vx}^2+{vy}^{2}+{vz}^{2}}
\end{align*}

Il y a risque de collision si et seulement si $t_{c} \geqslant 0$.

\begin{align*}
    t_{c} \geqslant  0  & \Leftrightarrow x \times vx + y \times vy + z \times vz \leqslant  0 \\
     t_{c} \geqslant  0   & \Leftrightarrow \overrightarrow{OG}(t_{0}) \, \dot \, \overrightarrow{V} \leqslant  0
\end{align*}


On en déduit qu'il n'y a pas de risque de collision si le produit scalaire $\overrightarrow{OG}(t_{0}) \, \dot \, \overrightarrow{V}$ est positif.


In [32]:
def calculer_CPA(intrus):
    
    def prodscal(v1, v2):
        return sum(c1*c2 for (c1, c2) in zip(v1, v2))
        
    (id, x, y, z, vx, vy, vz, t0) = intrus
    p = prodscal((x,y,z), (vx, vy, vz))
    if p > 0:
        return None
    tCPA =  -p/(vx**2 + vy**2 + vz**2)
    dt = tCPA - t0
    v = (x + dt*vx, y + dt*vy, z + dt*vz)
    dCPA = math.sqrt(prodscal(v, v))
    #conversion de zCPA en pieds
    zCPA = v[2]/0.3048  #un pied égal à 30,48 cm
    return [tCPA, dCPA, zCPA]

## Mise à jour de la liste des CPA 

In [33]:
def mettre_a_jour_CPAs(CPAs, id, nv_CPA, intrus_max, suivi_max):
    nbCPAs = len(CPAs)
    k = 0
    #recherche sequentielle de l'avion d'identifiant id
    #dans la liste CPAs    
    while k < nbCPAs and CPAs[k][0] != id:
        k += 1
    #si l'avion est  deja suivi
    if k < nbCPAs:
        if not nv_CPA or nv_CPA[0] > suivi_max:
            del CPAs[k]
            return None
        else:
            CPAs[k] = [id] + nv_CPA
            return k
    # sinon si l'avion n'est pas deja suivi
    elif  nv_CPA and nv_CPA[0] < suivi_max:
        if nbCPAs < intrus_max:
            CPAs.append([id] + nv_CPA)
        elif nv_CPA[0] < CPAs[-1][1]:
            CPAs[-1] = [id] + nv_CPA
        return nbCPAs - 1
    return None   

In [34]:
def replacer(ligne, CPAs):
    """Replace la ligne ligne à sa place dans l'ordre croissant
    des tCPA dans la liste CPAs"""
    nbCPAs = len(CPAs)
    if ligne > 0 and CPAs[ligne][1] < CPAs[ligne - 1][1] :
        direction = -1
        compare = lambda x, y : x > y
    else:
        direction = 1
        compare = lambda x, y : x < y
    element = CPAs[ligne]
    telement = element[1]
    k = ligne
    while 0 <= k + direction < nbCPAs and compare(CPAs[k + direction][1], telement):
        CPAs[k] = CPAs[k+direction]
        k = k + direction
    CPAs[k] = element

In [35]:
compare = lambda x, y : x > y
compare(13, 10)

True

In [36]:
# une liste de tests
CPAs = [ [ 11305708, 1462190400, 2450, -1000],
       [12416823, 1462190412, 2500, 500],
       [32675398, 1462190455, 2000, 800],
       [165465446, 1462190405, 1800, 400],
       [18743283, 1462190463, 2100, -200]]

In [37]:
replacer(3, CPAs)

In [38]:
CPAs

[[11305708, 1462190400, 2450, -1000],
 [165465446, 1462190405, 1800, 400],
 [12416823, 1462190412, 2500, 500],
 [32675398, 1462190455, 2000, 800],
 [18743283, 1462190463, 2100, -200]]

In [39]:
def enregistrer_CPA(intrus, CPAs, intrus_max, suivi_max):
    id = intrus[0]
    nv_CPA = calculer_CPA(intrus)
    k = mettre_a_jour_CPAs(CPAs, id, nv_CPA, intrus_max, suivi_max)
    if k != None:
        replacer(k, CPAs)

## Evaluation des paramètres généraux du système TCAS

In [40]:
vitesse = 900 #vitesse d'un avion en km/h
distance = 60 #distance de l'intrus en km
vrelative_intrus_max = 2*vitesse
#temps de collision minimum en secondes
temps_collision_min = distance/vrelative_intrus_max*3600
print(temps_collision_min)

120.0


La valeur standard de `suivi_max` est très proche (100 s) tout en étant légèrement
inférieure, l'intrus n'est donc pas immédiatement intégré à la liste CPA pour éviter de la surcharger inutilement. D'autres avions avec des tCPA plus proches seront surveillés en priorité.
D'après l'énoncé :

    La fonction acquerir_intrus détermine la position et la vitesse d’un intrus particulier. À chaque appel, elle
    s’intéresse à un avion différent dans le périmètre de surveillance du TCAS. Lorsque tous les intrus ont été
    examinés, l’appel suivant revient au premier intrus qui est toujours dans le périmètre.

Ainsi si l'intrus détecté à 60 km poursuit sa trajectoire, il devrait normalement être capté par la fonction `acquerir_intrus` et intégré s'il le faut à la liste CPAs.

In [41]:
vitesse_ascensionnelle = 1500 #en pieds par minute
distance = 500 #en pieds
temps = distance/(2*vitesse_ascensionnelle) #en minutes
print("Ils doivent maneuvrer {:.3f} secondes avant leur CPA".format(temps*60))

Ils doivent maneuvrer 10.000 secondes avant leur CPA


La consigne donnée dans l'énoncé à la question III D 3) est cohérente avec ce réultat. 

Les spécifications du système TCAS prévoient que la position de chaque intrus doit être vérifiée au
moins une fois par seconde. Elles limitent d’autre part le nombre d’intrus suivis à 30. En déduire le temps
maximum dont dispose la fonction TCAS pour exécuter une fois sa boucle.

La fonction TCAS doit effectuer sa boucle en au plus $\frac{1}{30}$ de seconde.

Quel est le facteur limitant la vitesse d’exécution de la fonction TCAS ? En déduire un ordre de grandeur
du temps minimum d’exécution d’une boucle. Est-il compatible avec les spécifications du système TCAS ?

Réponse : c'est la recherche séquentielle de l'intrus dans la liste CPAs
puis éventuellement l'insertion de la ligne de l'intrus à sa place dans la liste CPAs.
Dans les deux cas la complexité est linéaire d'ordre $O(intrusmax)$.
Une machine exécutant $10^6$ opérations par secondes mettre environ $2 \times 30 \times 10^{-6} \approx 10^{-4}$ secondes pour réaliser une boucle. $\frac{1}{30} \approx 3 \times 10^{-2}$ secondes donc il y a de la marge.


In [42]:
1/30

0.03333333333333333